In [2]:
import itertools
import json
import pandas as pd
import os

In [3]:
models = ["vit", "convnext", "xception"]
versions = ["1.4", "2.0", "2.1"]
squeezed = True

d = {}
for model, version in itertools.product(models, versions):
    file = f"{model}_{version}_squeezed_robustness.json"
    with open(file) as f:
        data = json.load(f)
        d[model + "_" + version] = data

    if squeezed:
        file = f"{model}_{version}_squeezed_square_partial_robustness.json"

        full_file = f"{model}_{version}_squeezed_square_robustness.json"
        if os.path.exists(full_file):
            file = full_file

        with open(file) as f:
            data = json.load(f)
            for key in data.keys():
                d[model + "_" + version][key] = data[key]

In [8]:
df = pd.DataFrame(d)

df = df[df.index.str.contains('_acc')]
df = df.rename(index=lambda x: x.replace("_acc", ""), inplace=False)
df = df.rename(index=lambda x: x.replace("linf_0.01568627450980392", "eps=4/255"))
df = df.rename(index=lambda x: x.replace("linf_0.03137254901960784", "eps=8/255"))

df = df.reindex(sorted(df.columns, key=lambda x: (x.split("_")[1], x.split("_")[0])), axis=1)
df = df.drop(["pad3", "pad10", "pad50", "colorjitter"])
df = df.rename(index=lambda x: x.replace("pad30", "pad"))

df = df.astype(float).round(2)

df.to_clipboard()
df


,convnext_1.4,vit_1.4,xception_1.4,convnext_2.0,vit_2.0,xception_2.0,convnext_2.1,vit_2.1,xception_2.1
1.4,97.44,89.08,99.76,86.46,86.92,93.10,85.94,86.84,90.66
2.0,69.10,89.56,61.21,89.40,90.28,97.82,88.93,90.12,98.13
2.1,69.38,89.28,61.96,89.02,89.94,97.76,89.04,90.29,98.14
pad,57.12,67.53,98.64,68.37,67.00,84.28,69.52,73.46,89.16
compress,89.80,88.93,81.19,89.36,90.16,94.86,89.04,90.18,93.95
grayscale,85.61,76.82,62.57,77.16,78.44,87.73,76.07,78.86,87.05
randomaffine,64.12,75.77,98.48,82.52,77.80,93.33,82.73,80.65,95.81
eps=4/255,1.04,50.00,6.85,49.24,57.46,9.34,47.98,58.22,7.74
eps=8/255,0.01,24.14,6.48,18.50,31.00,3.60,17.69,30.34,3.46
